<a href="https://colab.research.google.com/github/nzhinusoftcm/review-on-collaborative-filtering/blob/master/2.%20user-based%20collaborative%20filtering.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Download tools

In [1]:
import os

if os.path.exists("tools.zip") or os.path.exists("tools"):
    print('[INFO] Tools already exist')
else:
    print('[INFO] Download tools ...')
    !wget https://www.dropbox.com/s/fm1zn2bf08lgnun/tools.zip
    
    print('[INFO] unzip downloaded tools ...')
    !unzip tools.zip

[INFO] Tools already exist


In [2]:
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from tools.utils import load_movies, load_ratings, download_data

import pandas as pd
import numpy as np
import zipfile
import os

# Download MovieLen ratings

In [3]:
data = os.path.join('tools', 'ml-latest-small')

if os.path.exists(data):
    print('Data already exists ...')
    ratings_csv, movies_csv = os.path.join(data, 'ratings.csv'), os.path.join(data, 'movies.csv')
else:
    ratings_csv, movies_csv = download_data()

Data already exists ...


In [4]:
ratings, movies = load_ratings(ratings_csv), load_movies(movies_csv)

In [5]:
ratings.head()

,userid,itemid,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [6]:
movies.head()

,itemid,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


# Transform rating dataframe to matrix

In [7]:
def ratings_matrix(ratings):
    
    R = csr_matrix(pd.crosstab(ratings.userid, ratings.itemid, ratings.rating, aggfunc=sum).fillna(0).values)
    
    return R

In [8]:
R = ratings_matrix(ratings)

# Memory based collaborative filtering

Memory based collaborative filtering (CF) also known as nearest neighbors based CF makes recommendation based on similar behavious of users and items. There are two types of memory based CF : <b>user-based</b> and <b>item-based</b> CF. Both of these algorithm usually proceed in three stages :

1. Similarity computation (between users or items)
2. Rating prediction (using ratings of similar users or items)
3. Top-N recommendation


# 1. User-based Collaborative Filtering

## Idea

Let $u$ be the user for which we plan to make recommendations. 

1. Find other users whose past rating behavior is similar to that of $u$
2. Use their ratings on other items to predict what the current user will like

## Algorithm : user-to-user collaborative filtering

The entire process of user-to-user CF algorithm is described as follow <a href="https://romisatriawahono.net/lecture/rm/survey/information%20retrieval/Bobadilla%20-%20Recommender%20Systems%20-%202013.pdf">(J. Bobadilla et al. 2013)</a>: For an active user $u$,

<ol>
    <li> First identify the set $G_u$ of $k$ most similar users. $G_u$ is the group users similar to the active user $u$. The similarity between two users $u$ and $v$ can be measured by the cosine similarity measure as follows :

\begin{equation}
 w_{u,v}=\frac{\vec{r}_u \cdot \vec{r}_v}{\|\vec{r}_u\|_2 \ast \|\vec{r}_v\|_2} = \frac{\sum_{i\in I}r_{u,i}r_{v,i}}{\sqrt{\sum_{i\in I} (r_{u,i})^2}\sqrt{\sum_{i\in I} (r_{v,i})^2}}
\end{equation}

$w_{u,v}$ is the degree of similarity between users $u$ and $v$. This term is computed for all $v\in U$, where $U$ is the set of all users. There remains the question of how many neighbors to select. As experimented by <a href="https://dl.acm.org/doi/10.1145/3130348.3130372">(Herlocker et al. 1999)</a>, $k\in [20,50]$ is a reasonable starting point in many domains.
    </li>
    <li> Find the set $C$ of candidate items, purchased by the group and not purchased by the active user $u$. Candidate items have to be the most frequent items purchased by the group.
    </li>
    <li>Aggregate ratings of users in $G_u$ to make predictions for user $u$ on items he has not already purchased. Several aggregation approaches are often used such as <b>average, weighted sum, ajusted weighted sum</b>. By using weighted sum, the predicted rating of user $u$ on item $i$ denoted by $\hat{r}_{u,i}$ is computed as follow :

\begin{equation}
 \hat{r}_{u,i}=\bar{r}_u + \frac{\sum_{v\in G_u}(r_{v,i}-\bar{r}_v)\cdot w_{u,v}}{\sum_{v\in G_u}|w_{u,v}|}.
\end{equation}

Ratings of similar users are weighted by the corresponding similarity with the active user. Summation are made over all the users who have rated item $i$. Subtracting the user’s mean rating $\bar{r}_v$ compensates for differences in users’ use of the rating scale as some users will tend to give higher ratings than others <a href="https://dl.acm.org/doi/10.1561/1100000009">(Michael D. Ekstrand, <i>et al.</i> 2011)</a>. This prediction is made for all items $i \in C$ not purchased by user $u$.
    </li>
    <li>The Top-$N$ recommendations are obtained by choosing the $N$ items which provide most satisfaction to the user according to prediction.
    </li>
</ol>

### Step 1. Identify $G_u$, the set of $k$ users similar to an active user $u$

To find the $k$ most similar users to $u$, we use the cosine similarity and compute $w_{u,v}$ for all $v\in U$. Fortunately, libraries such as <i>scikit-learn (sklearn)</i> are very useful for such tasks :

1. First of all, we create a nearest neighbors model with sklearn through the function ```create_model()```. This function creates and fit a nearest neighbors model with user's ratings. ```n_neighbors=21``` define the number of neighbors to return. With $k=20$ neighbors, $|G_u|=21$ as $G_u$ contains $20$ similar users added to the active user $u$. That is why ```n_neighbors=21```. Each row $r_u$ of the rating matrix $R$ represents ratings of user $u$ on all items of the database. Missing ratings are replaced with $0.0$. 
```python
R[u,:] # uth row of the rating matrix R. Ratings of user u on all items in the database
```
2. Function ```nearest_neighbors()``` returns the most similar users to a given user $u$.

In [9]:
def create_model(R):
    
    # create the nearest neighbors model
    model = NearestNeighbors(metric='cosine', n_neighbors=21, algorithm='brute')
    
    # fit the model with ratings
    model.fit(R)
    
    return model

In [10]:
def nearest_neighbors(userid, model):
    """
    return users similar to a given user.
    
    :param
        - userid : id of the user for which we want to find nearest neighbors
        - model : nearest neighbors model
    
    :return
        - similarities : distances of the neighbors from the referenced user
        - neighbors : neighbors of the referenced user in decreasing order of similarities
    """
    
    similarities, neighbors = model.kneighbors(R[userid - 1,:])
    
    similarities = 1 - np.squeeze(similarities)
    neighbors = np.squeeze(neighbors) + 1
        
    return similarities[1:].tolist(), neighbors[1:].tolist()

Let's call functions ```create_model()``` and ```nearest_neighbors()``` to respectively create the $k$-NN model and compute the nearest neighbors for a given user

In [11]:
model = create_model(R)

In [12]:
similarities, neighbors = nearest_neighbors(1, model)

Now we know the 20 most similar users for each user in the database. The cell below accesses to the list of similar users of user 1

### Step 2. Find candidate items

The set $C$ of candidate items are the most frequent ones purchased by users in $G_u$ for an active user $u$ and not purchased by $u$.

Function ```find_candidate_items()``` : find items purchased by these similar users as well as their frequency. Note that the frequency of the items in the set $C$ can be computed by just counting the actual occurrence frequency of that items.

1. ```Gu_items``` : frequent items of $G_u$ in decreasing order of frequency.
2. ```active_items``` : items already purchased by the active user
3. ```candidates``` : frequent items of $G_u$ not purchased by the active user $u$

In [13]:
def find_candidate_items(userid, neighbors):
    """
    Find candidate items for an active user
    
    :param
        - userid : active user
        - neighbors : users similar to the active user
        
    :return
        - candidates : top 30 of candidate items
    """
    
    activities = ratings.loc[ratings.userid.isin(neighbors)]
    
    # sort items in decreasing order of frequency
    frequency = activities.groupby('itemid')['rating'].count().reset_index(name='count').sort_values(['count'],ascending=False)
    Gu_items = frequency.itemid
    active_items = ratings.loc[ratings.userid == userid].itemid.to_list()
    candidates = np.setdiff1d(Gu_items, active_items, assume_unique=True)[:30]
        
    return candidates

### Step 3. Rating prediction

Now it's time to predict what score the active user $u$ would have given to each of the top-30 candidate items.

To predict the score of $u$ on a candidate item $i$ ,we need :
1. Similarities between $u$ and all his neighbors $v \in G_u$ who rated item $i$ : function ```nearest_neighbors()``` returns similar users of a user as well as their corresponding similarities.
2. Normalized ratings of all $v \in G_u$ on item $i$. The normalized rating of user $v$ on item $i$ is defined by $r_{v,i}-\bar{r}_v$.

Next, let's compute the mean rating of each user and the normalized ratings for each item. The DataFrame ```mean``` contains mean rating for each user. With the mean rating of each user, we can add an extra column ```norm_rating``` to the ```ratings```'s DataFrame which can be accessed to make predictions.

In [14]:
# mean ratings for each user
mean = ratings.groupby(by='userid', as_index=False)['rating'].mean()
mean_ratings = pd.merge(ratings, mean, suffixes=('','_mean'), on='userid')

# normalized ratings for each items
mean_ratings['norm_rating'] = mean_ratings['rating'] - mean_ratings['rating_mean']

In [15]:
mean_ratings

,userid,itemid,rating,rating_mean,norm_rating
0,1,1,4.0,4.366379,-0.366379
1,1,3,4.0,4.366379,-0.366379
2,1,6,4.0,4.366379,-0.366379
3,1,47,5.0,4.366379,0.633621
4,1,50,5.0,4.366379,0.633621
...,...,...,...,...,...
100831,610,166534,4.0,3.688556,0.311444
100832,610,168248,5.0,3.688556,1.311444
100833,610,168250,5.0,3.688556,1.311444
100834,610,168252,5.0,3.688556,1.311444


Find similar users who rated the random item. The datafram ```urci``` correspond to <b>users who rated the current item</b> and ```surci``` to the <b> similar users who rated the current item</b>

In [16]:
def surci(itemid, neighbors):
    """
    """
    urci = mean_ratings[mean_ratings.itemid==random_item]
    surci = URCI.loc[URCI.userid.isin(neighbors)]
    
    return surci

The active user then has eleven (10) similar users who rated the selected item. We will use their ratings to predict what he would have given to that item. Recall that the prediction formula is defined as follow :

\begin{equation}
 \hat{r}_{u,i}=\bar{r}_u + \frac{\sum_{v\in G_u}(r_{v,i}-\bar{r}_v)\cdot w_{u,v}}{\sum_{v\in G_u}|w_{u,v}|}.
\end{equation}

In [17]:
def rating_prediction(userid, itemid, similarities, neighbors):
    """
    predict what score userid would have given to itemid.
    
    :param
        - userid : user id for which we want to make prediction
        - itemid : item id on which we want to make prediction
        
    :return
        - r_hat : predicted rating of user userid on item itemid
    """
    
    # get mean rating of user userid
    user_mean = mean[mean.userid==userid].rating.values[0]
    
    # initialize the weighted sum (numerator)
    weighted_sum = 0
    
    # initialize the sum of similarities of neighbors who rated item itemid (denominator)
    W = 0
    
    # users who rated current item
    URCI = mean_ratings[mean_ratings.itemid==itemid]
    
    # similar users who rated the current item
    SURCI = URCI.loc[URCI.userid.isin(neighbors)]
    
    # loop over neighbors who rated item item
    for uid in SURCI.userid:
        
        # similarity between userid an uid
        w = similarities[neighbors.index(uid)]
        
        # normalize rating of uid on itemid
        norm_score = SURCI[SURCI.userid==uid].norm_rating.values[0]
        
        # weighted normalize rating
        weighted_score = norm_score * w
        
        # update denominator
        W = W + abs(w)
        
        # update denominator
        weighted_sum = weighted_sum + weighted_score
    
    r_hat = user_mean + weighted_sum / W
    
    return r_hat

Now, we can make rating prediction for a given user on each item in his set of candidate items.

In [18]:
def user2userPredictions(userid, pred_path):
    """
    Make rating prediction for the active user on each candidate item and save in file prediction.csv
    
    :param
        - userid : id of the active user
        - pred_path : where to save predictions
    """
    similarities, neighbors = nearest_neighbors(userid, model)
    
    # find candidate items for the active user
    candidates = find_candidate_items(userid, neighbors)
    
    # loop over candidates items to make predictions
    for itemid in candidates:
        
        # prediction for userid on itemid
        r_hat = rating_prediction(userid, itemid, similarities, neighbors)
        
        # save predictions
        with open(pred_path, 'a+') as file:
            line = '{},{},{}\n'.format(userid, itemid, r_hat)
            file.write(line)

In [19]:
def user2userCF():
    """
    Make predictions for each user in the database.    
    
    """
    # get list of users in the database
    users = ratings.userid.unique()
    
    saved_predictions = os.path.join('tools', 'predictions.csv')    
    if os.path.exists(saved_predictions):
        os.remove(saved_predictions)
    
    for userid in users:        
        
        # make rating predictions for the current user
        user2userPredictions(userid, saved_predictions)

In [20]:
user2userCF()

### Step 4. Top-N recommendation

Function ```user2userRecommendation()``` reads predictions for a given user and return the list of items in decreasing order of predicted rating.

In [21]:
def user2userRecommendation(userid):
    """
    """
    saved_predictions = os.path.join('tools', 'predictions.csv')
    
    predictions = pd.read_csv(saved_predictions, sep=',', names=['userid', 'itemid', 'predicted_rating'])
    predictions = predictions[predictions.userid==userid]
    List = predictions.sort_values(by=['predicted_rating'], ascending=False)
    List = pd.merge(List, movies, on='itemid', how='inner')
    
    return List

In [22]:
user2userRecommendation(212)

,userid,itemid,predicted_rating,title
0,212,50,4.363048,"Usual Suspects, The (1995)"
1,212,858,4.336386,"Godfather, The (1972)"
2,212,527,4.300788,Schindler's List (1993)
3,212,48516,4.265758,"Departed, The (2006)"
4,212,47,4.210786,Seven (a.k.a. Se7en) (1995)
5,212,4878,4.186514,Donnie Darko (2001)
6,212,1089,4.172989,Reservoir Dogs (1992)
7,212,1270,4.112967,Back to the Future (1985)
8,212,4226,4.047615,Memento (2000)
9,212,1,4.004465,Toy Story (1995)


## Limitations of user-based CF

1. <b> Sparsity </b> : In general, users interact with less than 20% of items. This leads the rating matrix to be highly sparse. For example, the movielen-100k contains 100k ratings from 943 users on 1682 items. The pourcentage of sparsity in this case is around $94\%$. A recommender system based on nearest neighbor algorithms may be unable to make any item recommendations for a particular user. As a result the accuracy of recommendations may be poor <a href="https://dl.acm.org/doi/10.1145/371920.372071">(Sarwar <i>et al.</i> 2001)</a>.

2. <b> Stability of user's ratings </b> : As a user rates and re-rates items, their rating vector will change along with their similarity to other users. A user’s neighborhood is determined not only by their ratings but also by the ratings of other users, so their neighborhood can change as a result of new ratings supplied by any user in the system <a href="https://dl.acm.org/doi/10.1561/1100000009">(Michael D. Ekstrand, <i>et al.</i> 2011)</a>.

3. <b> Scalability </b> : Due to the non-stability of users ratings, finding similar users in advance is complicated. For this reason, most user-based CF systems find neighborhoods each time predictions or recommendations are needed. However, these are huge computations that grows with both the number of users and the number of items. With millions of users and items, a typical web-based recommender system running existing algorithms will suffer serious scalability concerns <a href="https://dl.acm.org/doi/10.1145/371920.372071">(Sarwar <i>et al.</i> 2001)</a>, <a href="https://dl.acm.org/doi/10.1561/1100000009">(Michael D. Ekstrand, <i>et al.</i> 2011)</a>.

## References

1. Herlocker et al. (1999)<a href="https://dl.acm.org/doi/10.1145/3130348.3130372"> An Algorithmic Framework for Performing Collaborative Filtering</a>
2. Sarwar et al. (2001) <a href="https://dl.acm.org/doi/10.1145/371920.372071"> Item-based collaborative filtering recommendation algorithms</a> 
3. Michael D. Ekstrand, et al. (2011). <a href="https://dl.acm.org/doi/10.1561/1100000009"> Collaborative Filtering Recommender Systems</a>
4. J. Bobadilla et al. (2013)<a href="https://romisatriawahono.net/lecture/rm/survey/information%20retrieval/Bobadilla%20-%20Recommender%20Systems%20-%202013.pdf"> Recommender systems survey</a>

## Author

<a href="https://www.linkedin.com/in/carmel-wenga-871876178/">Carmel WENGA</a>, Applied Machine Learning Research Engineer | <a href="https://shoppinglist.cm/fr/">ShoppingList</a>, Nzhinusoft